# Compute statistics about the global database

* use lprofiles, ltags, ldeployments
* list files are created in sc_list_profiles.ipynb

In [1]:
from pathlib import Path
import sys
import os
import shutil
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import gsw
from IPython.display import display,Image

from importlib import reload
import meop_metadata
import meop_plot_data
reload(meop_metadata)

<module 'meop_metadata' from '/media/disk2/roquet/MEOP_process/python/meop_metadata.py'>

## Write statistics

In [3]:
lprofiles, ltags, ldeployments = meop_metadata.read_lists_metadata(rebuild=False,verbose=False,public=False,Tdata=True)
lprofiles_public, ltags_public, ldeployments_public = meop_metadata.read_lists_metadata(rebuild=False,verbose=False,public=True,Tdata=True)

In [4]:
lprofiles_public

,DEPLOYMENT_CODE,SMRU_PLATFORM_CODE,CYCLE_NUMBER,JULD,LATITUDE,LONGITUDE,N_TEMP,N_PSAL,N_CHLA,year,month,day,MASK
0,wd04,wd04-907-11,1,2011-03-25 09:30:00,-68.430260,78.334786,27.0,NaN,NaN,2011,3,25,Southern Ocean
1,wd04,wd04-907-11,2,2011-03-25 14:00:00,-68.408180,78.434688,6.0,NaN,NaN,2011,3,25,Southern Ocean
2,wd04,wd04-907-11,3,2011-03-25 20:49:59,-68.360050,78.560489,28.0,NaN,NaN,2011,3,25,Southern Ocean
3,wd04,wd04-907-11,4,2011-03-26 02:09:59,-68.372521,78.528017,28.0,NaN,NaN,2011,3,26,Southern Ocean
4,wd04,wd04-907-11,5,2011-03-26 05:20:00,-68.395222,78.509480,28.0,NaN,NaN,2011,3,26,Southern Ocean
...,...,...,...,...,...,...,...,...,...,...,...,...,...
644375,ct83,ct83-859-13,337,2014-06-19 20:29:59,-62.324400,-60.801100,16.0,16.0,NaN,2014,6,19,Southern Ocean
644376,ct83,ct83-859-13,338,2014-06-20 03:10:00,-62.323200,-60.946100,16.0,16.0,NaN,2014,6,20,Southern Ocean
644377,ct83,ct83-859-13,339,2014-06-20 12:10:00,-62.289400,-60.876400,16.0,16.0,NaN,2014,6,20,Southern Ocean
644378,ct83,ct83-859-13,340,2014-06-21 03:59:59,-62.353100,-60.765100,8.0,8.0,NaN,2014,6,21,Southern Ocean


In [5]:
ltags.columns

Index(['SMRU_PLATFORM_CODE', 'DEPLOYMENT_CODE', 'JULD', 'LATITUDE',
       'LONGITUDE', 'year_x', 'month', 'day', 'MASK', 'N_PROF_TEMP',
       'N_PROF_PSAL', 'N_PROF_CHLA', 'PUBLIC', 'T1', 'T2', 'S1', 'S2',
       'remove', 'Sremove', 'comments', 'variable_offset', 'instr_id',
       'year_y', 'period', 'continuous', 'prefix'],
      dtype='object')

In [6]:
print('Number of valid T profiles, total :',lprofiles.N_TEMP.count())
print('Number of valid T profiles, public:',lprofiles_public.N_TEMP.count())
print('Number of valid T profiles, public, south of 55S: ',lprofiles_public[lprofiles_public.LATITUDE<-55].N_TEMP.count())

Number of valid T profiles, total : 656134
Number of valid T profiles, public: 590379
Number of valid T profiles, public, south of 55S:  272602


In [11]:
print('Number of valid S profiles, total :',lprofiles.N_PSAL.count())
print('Number of valid S profiles, public:',lprofiles_public.N_PSAL.count())
print('Number of valid S profiles, public, south of 55S: ',lprofiles_public[lprofiles_public.LATITUDE<-55].N_PSAL.count())

Number of valid S profiles, total : 521754
Number of valid S profiles, public: 457743
Number of valid S profiles, public, south of 55S:  234952


In [17]:
# select tags with at least one profile south of 55S
index_tags = lprofiles_public[lprofiles_public.LATITUDE<-55].SMRU_PLATFORM_CODE.unique()
print('Number of valid tags, public, south of 55S: ',len(index_tags))

Number of valid tags, public, south of 55S:  781


In [9]:
# select tags with at least one Chla profile
index_tags = lprofiles_public[lprofiles_public.N_CHLA>0].SMRU_PLATFORM_CODE.unique()
print('Number of valid tags, public, with chla: ',len(index_tags))

# select tags with at least one Chla profile south of 50S
index_tags = lprofiles_public[(lprofiles_public.N_CHLA>0)&(lprofiles_public.LATITUDE<-55)].SMRU_PLATFORM_CODE.unique()
print('Number of valid tags, public, with chla: ',len(index_tags))

# Number of profiles of Chla
print('')
print('Number of valid F profiles, total : ',lprofiles[lprofiles.N_CHLA>0].N_CHLA.count())
print('Number of valid F profiles, public: ',lprofiles_public[lprofiles_public.N_CHLA>0].N_CHLA.count())
print('Number of valid F profiles, public, south of 55S: ',lprofiles_public[(lprofiles_public.N_CHLA>0)&(lprofiles_public.LATITUDE<-55)].N_CHLA.count())

Number of valid tags, public, with chla:  108
Number of valid tags, public, with chla:  26

Number of valid F profiles, total :  26801
Number of valid F profiles, public:  26328
Number of valid F profiles, public, south of 55S:  3528
